In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
import talib
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model

# Tải mô hình đã huấn luyện
model = load_model('/home/phung/jnotebook/source/Tuan10/lstm_stock_model.keras')


2025-03-20 10:14:33.340543: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
stock = 'GOOG'
start = '2010-01-01'
end = datetime.today().strftime('%Y-%m-%d')
data = yf.download(stock, start, end)

if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.droplevel(1)

# 2. Tính toán các chỉ báo kỹ thuật
data['SMA_50'] = talib.SMA(data['Close'].values, timeperiod=50)
data['EMA_20'] = talib.EMA(data['Close'].values, timeperiod=20)
data['RSI_14'] = talib.RSI(data['Close'].values, timeperiod=14)
data['MACD'], data['MACD_signal'], _ = talib.MACD(data['Close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
data['Upper_BB'], data['Middle_BB'], data['Lower_BB'] = talib.BBANDS(data['Close'].values, timeperiod=20, nbdevup=2, nbdevdn=2)
data['ADX_14'] = talib.ADX(data['High'].values, data['Low'].values, data['Close'].values, timeperiod=14)

# Xóa dữ liệu NaN do tính toán chỉ báo
data.dropna(inplace=True)
data.reset_index(inplace=True)
# Chọn các đặc trưng đầu vào
features = ['Close', 'SMA_50', 'EMA_20', 'RSI_14', 'MACD', 'MACD_signal', 'Upper_BB', 'Lower_BB', 'ADX_14']
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data[features])

[*********************100%***********************]  1 of 1 completed


In [15]:
sequence_length = 100
input_data = data_scaled[-sequence_length:]

# Chuyển đổi thành dạng phù hợp với mô hình
x_input = np.array([input_data])  # Thêm batch dimension

# Dự đoán giá cho 7 ngày tiếp theo
predicted_prices = []
for _ in range(45):
    pred = model.predict(x_input)[0, 0]
    predicted_prices.append(pred)

    # Cập nhật input_data với giá dự đoán mới
    new_row = np.append(x_input[0, 1:], [[pred] + [0] * (len(features) - 1)], axis=0)
    x_input = np.array([new_row])

# Chuyển đổi về giá trị gốc
scaler_close = MinMaxScaler()
scaler_close.fit(data[['Close']])
predicted_prices = scaler_close.inverse_transform(np.array(predicted_prices).reshape(-1, 1))

# In kết quả dự đoán
print("Dự đoán giá cổ phiếu trong 7 ngày tới:")
for i, price in enumerate(predicted_prices, 1):
    print(f"Ngày {i}: {price[0]:.2f} USD")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━